In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from airfoil.airfoil import Airfoil, Hole
from shapely.plotting import plot_polygon, plot_line
from util.path_planning import split_and_roll, intro_path

In [ ]:
af = Airfoil.from_naca_designation("NACA 2414", 100,50)
ax,_=af.plot()

In [ ]:
import numpy as np
def zigzag(minx,miny,maxx, maxy, peaks:int=20):
    x = np.linspace(0, 1, peaks)
    y = np.zeros_like(x)
    y[::2]=1
    rx = (maxx-minx)*x+minx
    ry = (maxy-miny)*y+miny
    return np.vstack([rx,ry]).transpose()

In [ ]:
af.plot()

In [ ]:
from shapely import box, intersection, difference, union, LineString, Polygon
afp = af.polygon()
afpb = afp.bounds
trussed = union(
    intersection(
        afp,
        LineString(zigzag(*afpb,10)).buffer(2)
    ),
    difference(
        afp,
        intersection(
            afp.buffer(-2),
            box(15,-300,80,300)
        )
    )
)
top    = trussed.intersection(box(-500,0,500,500)).exterior
bottom = trussed.intersection(box(-500,-500,500,0)).exterior
#plot_line(bottom)

In [ ]:
plot_line(top)
plot_line(bottom)

In [ ]:
path = intro_path(split_and_roll(Polygon(bottom)), padding=5)
path=np.concat([
    path,
    [[path[-1][0]+30,path[-1][1]]]
])
plt.plot(*path.T)
plt.gca().set_aspect("equal")

In [ ]:
from util.serial import CNC

In [ ]:
cnc = CNC()

In [ ]:
cnc.home()

In [ ]:
cnc.writeln("G90 G21 F1000 X104 Y10 Z104 A10")

In [ ]:
cnc.writeln("G92 X0 Y0 Z0 A0")

In [ ]:
cnc.absolute()

In [ ]:
cnc.send_g1_xy_commands(path, feed=300)

In [ ]:
cnc.writeln("G90 G21 F1000 X0 Y3 Z0 A3")

In [ ]:
cnc.writeln("G92 X0 Y0 Z0 A0")